We're going to use the Generator and discriminators to thus train on the LJSpeech dataset. The dataset consists of around 13,000 samples reading from 7 non-fiction books. They vary from 1 - 10 seconds and total around 24h.  



In [ ]:
import os
import librosa
import numpy as np
import soundfile as sf

def wav_to_mel(wav_path, mel_path=80, sr=22050, n_fft=1024, hop_size=256, win_length=1024, n_mels=128, n_freq=1024, fmin=0, fmax=8000):
    # Load the audio file
    y, _ = librosa.load(wav_path, sr=sr)
    
    # Compute the Mel spectrogram
    mel_spectrogram = librosa.feature.melspectrogram(y=y, sr=sr, n_fft=n_fft, hop_size=hop_size, win_length=win_length, n_mels=n_mels, n_freq=n_freq, fmin=fmin, fmax=fmax)
    
    # Convert to decibels
    mel_spectrogram_db = librosa.power_to_db(mel_spectrogram, ref=np.max)
    
    # Save the Mel spectrogram as a numpy file
    np.save(mel_path, mel_spectrogram_db)

def process_dataset(dataset_path):
    # Create output directories for Mel spectrograms and wave files
    mel_dir = os.path.join(dataset_path, 'mel_spectrograms')
    wav_dir = os.path.join(dataset_path, 'processed_wavs')
    os.makedirs(mel_dir, exist_ok=True)
    os.makedirs(wav_dir, exist_ok=True)

    # Process each WAV file in the dataset_path
    for filename in os.listdir(dataset_path):
        if filename.endswith('.wav'):
            wav_path = os.path.join(dataset_path, filename)
            mel_path = os.path.join(mel_dir, filename.replace('.wav', '.npy'))
            processed_wav_path = os.path.join(wav_dir, filename)
            
            # Transform WAV to Mel spectrogram
            wav_to_mel(wav_path, mel_path)
            
            # Optionally, you can copy or modify the wav file
            # For now, let's just copy it to the processed_wavs directory
            sf.write(processed_wav_path, librosa.load(wav_path)[0], 22050)


In [ ]:
import argparse
import jax
import os

from Generator import Generator
from Discriminators import MultiPeriodDiscriminator, MultiScaleDiscriminator, feature_loss, generator_loss, discriminator_loss

def create_parser():
    parser = argparse.ArgumentParser(description="Arguments for training HiFiGaN")

    parser.add_argument("--dataset_path", "Path to the dataset to use (LJSpeech)")

    parser.add_argument("--learning_rate", "Learning rate during training")

    parser.add_argument("--output_path", "Path to store model weights")

    return parser




def train_hifigan(dataset_path, output_path, learning_rate):
    key = jax.random.PRNG()

    key1, key2, key3 = jax.random.split(key, 3)

    generator = Generator(channels_in=80, channels_out=1, key=key1)
    scale_disc = MultiScaleDiscriminator(key=key2)
    period_disc = MultiPeriodDiscriminator(key=key3)

    dataset = 


if __name__ == "__main__":
    parser = create_parser()
    args = parser.parse_args()

    train_hifigan(dataset_path=args.dataset_path, output_path=args.output_path, learning_rate=args.learning_rate)
